In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.utils import shuffle
from sklearn import datasets
from sklearn.svm import SVR
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [2]:
t_df = pd.read_pickle("dataSet/t_df.pkl")

X = t_df.drop("survived", axis = 1)
y = t_df["survived"]

X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [3]:
k_model = KNeighborsClassifier(n_neighbors = 5)
k_model.fit(X_train, y_train)
k_pred = k_model.predict(X_test)
k_pred_proba = k_model.predict_proba(X_test)

C:\Users\h\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [4]:
def get_clf_eval(y_test = None, pred = None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = round(accuracy_score(y_test, pred),4)
    precision = round(precision_score(y_test, pred),4)
    recall = round(recall_score(y_test, pred),4)
    f1 = round(f1_score(y_test, pred),4)
    roc_auc = round(roc_auc_score(y_test, pred_proba[:, 1]),4)
    print("오차 행렬")
    print(confusion)
    print(f"정확도 : {accuracy}, 정밀도 : {precision}\n재현율 : {recall}, f1 : {f1}")
    print(f"AUC : {roc_auc}")

In [5]:
get_clf_eval(y_test, k_pred, k_pred_proba)

오차 행렬
[[140  18]
 [ 43  61]]
정확도 : 0.7672, 정밀도 : 0.7722
재현율 : 0.5865, f1 : 0.6667
AUC : 0.7889


### 서포트 벡터 머신(Support Vector Machines, SVM)

- 분류나 회귀, 이상치 탐지 등에 사용되는 강력한 머신러닝 알고리즘 중 하나입니다. SVM은 주로 분류 문제에 사용되며, 이 알고리즘의 핵심 아이디어는 데이터를 고차원 공간으로 변환하여 서로 다른 클래스 간의 최대 마진을 찾는 것입니다.

- SVM은 데이터를 두 개의 클래스로 나누는 결정 경계(결정 초평면이라고도 함)를 찾습니다. 이 결정 경계는 각 클래스의 가장 가까운 훈련 샘플(서포트 벡터라고 함)까지의 거리가 최대가 되는 선을 찾는 것을 목표로 합니다. 이를 '마진 최대화'라고 하며, 이 마진 최대화는 오류를 최소화하고 모델의 일반화 성능을 향상시키는 데 중요한 역할을 합니다.

- SVM은 선형 뿐만 아니라 비선형 분류 문제에도 사용할 수 있습니다. 비선형 문제를 해결하기 위해, SVM은 커널 트릭이라는 기법을 사용하여 데이터를 고차원 공간으로 변환하고, 그 고차원에서 선형 결정 경계를 찾습니다. 이 커널 트릭 덕분에 SVM은 복잡한 분류 문제를 처리할 수 있습니다.

- SVM은 작은 데이터셋에서도 잘 작동하며, 높은 차원의 데이터에 대해 강력한 성능을 발휘합니다. 그러나 데이터셋이 크거나 노이즈가 많은 경우, 그리고 데이터가 선형적으로 구분되지 않는 경우에는 다른 알고리즘(예: 랜덤 포레스트나 신경망)에 비해 성능이 떨어질 수 있습니다.

In [6]:
t_df = pd.read_pickle("dataSet/t_df.pkl")

X = t_df.drop("survived", axis = 1)
y = t_df["survived"]

X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test, train_test_split(X, y, test_size = 0.2, random_state = 10)

(array([[ 8.41916418e-01,  7.43496915e-01, -4.56671360e-01,
          6.22279324e-01, -5.58346054e-01],
        [-1.54609786e+00, -1.34499549e+00, -1.00022257e+00,
         -1.83492621e+00,  7.33522906e-02],
        [-1.54609786e+00, -1.34499549e+00,  6.30431068e-01,
         -1.83492621e+00,  7.33522906e-02],
        ...,
        [-3.52090723e-01,  7.43496915e-01, -1.46070666e-01,
          6.22279324e-01, -5.58346054e-01],
        [ 8.41916418e-01, -1.34499549e+00, -5.51737667e-16,
         -6.06323444e-01, -5.58346054e-01],
        [ 8.41916418e-01,  7.43496915e-01, -6.89621880e-01,
          6.22279324e-01,  7.33522906e-02]]),
 array([[-1.54609786e+00,  7.43496915e-01, -4.56671360e-01,
          6.22279324e-01,  7.33522906e-02],
        [-3.52090723e-01,  7.43496915e-01, -1.46070666e-01,
          6.22279324e-01, -5.58346054e-01],
        [ 8.41916418e-01,  7.43496915e-01, -5.51737667e-16,
         -1.83492621e+00, -5.58346054e-01],
        ...,
        [ 8.41916418e-01,  7.4349691

### 커널 함수의 역할

- 차원의 확장: 데이터가 선형적으로 구분되지 않는 경우, 그 데이터를 더 높은 차원으로 매핑하여 선형적으로 분리할 수 있게 하는 것이 커널 함수의 주요 기능입니다. 예를 들어, 2차원 공간에서는 선형적으로 분리할 수 없는 데이터를 3차원 공간으로 매핑함으로써, 선형 분리가 가능해질 수 있습니다.

- 컴퓨터 연산의 효율성: 커널 함수를 사용하면 실제로 데이터를 더 높은 차원으로 변환하는 복잡한 계산을 수행할 필요 없이, 원래의 차원에서 두 데이터 포인트의 유사도를 계산함으로써 동일한 효과를 얻을 수 있습니다. 이것을 '커널 트릭'이라고도 합니다. 이 방법은 컴퓨터 메모리와 연산 속도를 크게 절약할 수 있습니다.

- 비선형 분리 가능: 실제 세계의 많은 데이터셋들은 선형적으로 완벽하게 분리할 수 없습니다. 커널 함수를 사용하면 이러한 비선형 데이터셋도 잘 분리할 수 있습니다.

- 커널 함수는 여러 형태가 있는데, 선형 커널, 다항 커널, RBF(Radial Basis Function) 커널, 시그모이드 커널 등이 있습니다. 이들 중에서 가장 널리 사용되는 것은 RBF 커널입니다.

In [7]:
svm_model = svm.SVC(kernel = "rbf", random_state = 0)
svm_model.fit(X_train, y_train)
s_pred = svm_model.predict(X_test)
print(round(accuracy_score(y_test, s_pred),4))

0.8092


### 서포트 벡터 회귀(Support Vector Regression, SVR)
- 서포트 벡터 머신(SVM)의 개념을 회귀 문제에 적용한 것입니다.

- SVM이 두 클래스 사이의 마진을 최대화하는 방식으로 분류 문제를 해결하는 것과 달리, SVR은 이 마진 안에 가능한 한 많은 데이터 포인트가 포함되도록 회귀선을 조정합니다. 이 마진은 사용자가 정의한 오차 허용 범위(epsilon)에 의해 결정되며, 이 범위 안의 오차는 모델에게 패널티를 주지 않습니다.

- SVR의 목표는 주어진 epsilon에 대해 최대한 많은 데이터 포인트가 떨어져 있는 회귀선을 찾는 것입니다. 이 회귀선은 최대한 많은 데이터 포인트를 epsilon 범위 안에 포함하려고 시도하며, 범위 밖의 데이터 포인트는 비용 함수를 통해 패널티를 부여받습니다.

- SVR은 선형 회귀뿐만 아니라 비선형 회귀 문제에도 적용될 수 있습니다. 비선형 문제에서는 커널 트릭을 사용하여 데이터를 고차원으로 매핑하고, 이러한 고차원 공간에서 선형 회귀를 수행합니다.

- SVR은 모델의 복잡성을 제어하는 데 중요한 두 가지 매개변수, 즉 C와 epsilon을 가지고 있습니다. C는 패널티 항의 강도를 제어하고, epsilon은 마진의 폭을 제어합니다. 이 매개변수들은 모델의 성능과 과적합 문제를 제어하는 데 중요한 역할을 합니다.

In [8]:
data = datasets.load_boston()

X, y = shuffle(data.data, data.target, random_state = 7)

num_training = int(0.8 * len(X))

X_train, y_train = X[:num_training], y[:num_training]
X_test, y_test = X[num_training:], y[num_training:]

sv_regressor = SVR(kernel = "linear", C = 1, epsilon = 0.1)

sv_regressor.fit(X_train, y_train)

y_pred = sv_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
evs = explained_variance_score(y_test, y_pred)

print(f"mse : {mse:.3f}")
print(f"rmse : {rmse:.3f}")

test_data = [3.7, 0, 18.4, 1, 0.87, 5.96, 81, 2.5052, 26, 656, 20.2, 351.34, 15.27]
print("예측값 : ", np.round(sv_regressor.predict([test_data])[0],3))

C:\Users\h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

mse : 15.377
rmse : 3.921
예측값 :  19.0


### 로지스틱 회귀(Logistic Regression)

- 분류 문제를 해결하기 위한 알고리즘으로, 선형 회귀와 마찬가지로 입력 특성의 가중치 합을 계산하는데, 선형 회귀와 달리 결과를 이진 분류(0 또는 1, 참 또는 거짓 등)에 사용할 수 있는 확률로 변환합니다.

- 로지스틱 회귀는 선형 회귀와 비슷하게 모델의 예측과 실제 값 사이의 차이를 최소화하도록 가중치를 학습합니다. 하지만 로지스틱 회귀는 선형 회귀와는 달리 결과를 0과 1 사이의 값으로 제한하는 로지스틱 함수(또는 시그모이드 함수)를 사용합니다.

- 로지스틱 회귀는 각 클래스에 속할 확률을 제공하며, 특정 임계값(일반적으로 0.5)을 초과하는 경우 데이터 포인트를 해당 클래스에 할당합니다. 이는 이진 분류뿐만 아니라 다중 클래스 분류에도 적용될 수 있습니다(이 경우에는 일대다(OvR) 또는 다항 로지스틱 회귀를 사용할 수 있습니다).

- 로지스틱 회귀는 출력이 확률이기 때문에, 결과의 해석이 직관적이며 모델의 예측이 불확실한 경우에도 그 정도를 측정할 수 있습니다. 또한 로지스틱 회귀는 선형 회귀보다 이상치에 덜 민감하며, 모델이 과적합되는 것을 방지하기 위해 규제를 적용할 수 있다는 장점도 있습니다.

- 로지스틱 함수, 또는 시그모이드 함수는 S-자 형태를 띠는 함수로, 실수 입력값을 0과 1 사이의 출력값으로 변환하는 데 사용됩니다. 이 함수는 머신러닝, 특히 이진 분류 문제에서 중요한 역할을 합니다.

- 로지스틱 함수의 정의

    - f(x) = 1 / (1 + e^-x)

    - e는 자연 상수(약 2.71828)입니다. x는 어떤 실수 값도 가능하며, -무한대에서 무한대까지의 범위를 가집니다.

    - 함수가 결과를 0과 1 사이로 제한하기 때문에, 이는 확률에 대해 논의할 때 특히 유용합니다. 로지스틱 회귀 분석에서 이 함수는 선형 함수의 결과를 확률로 변환하는데 사용됩니다.

    - 입력값 x가 커질수록 로지스틱 함수의 출력은 1에 가까워지고, x가 작아질수록 출력은 0에 가까워집니다. x가 0일 때 로지스틱 함수의 값은 0.5입니다. 이러한 특성 때문에 로지스틱 함수는 이진 분류 문제에 널리 사용됩니다.

In [9]:
cancer = datasets.load_breast_cancer()

scaler = StandardScaler()
data_scaled = scaler.fit_transform(cancer.data)

X_train, X_test, y_train, y_test = train_test_split(data_scaled, cancer.target, test_size = 0.3, random_state = 0)

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

print(f"accuracy : {accuracy_score(y_test, lr_preds):.4f}")
print(f"roc_auc : {roc_auc_score(y_test, lr_preds):.4f}")

accuracy : 0.9766
roc_auc : 0.9716


### 주요 하이퍼파라미터

- Regularization parameter (C): 로지스틱 회귀에서는 과적합을 방지하기 위해 정규화(regularization)를 사용합니다. 정규화는 모델의 복잡성에 대한 패널티를 부여하며, 이때 C는 이 패널티의 강도를 조정하는 역할을 합니다. C 값이 크면 패널티가 약해져 모델은 복잡해질 수 있으며, 이는 과적합의 가능성을 높일 수 있습니다. 반대로 C 값이 작으면 패널티가 강해져 모델이 단순해질 수 있으며, 이는 과소적합의 가능성을 높일 수 있습니다.

- Penalty: Penalty는 정규화의 형태를 결정하는 하이퍼파라미터입니다. 주로 L1, L2 두 가지 형태가 사용됩니다. L1 정규화는 변수의 수를 줄여 희소한 모델을 만드는데 효과적인 반면, L2 정규화는 모델의 가중치를 균등하게 줄여 과적합을 방지하는 데 효과적입니다. 이 두 정규화 방법은 로지스틱 회귀 모델의 성능과 특성을 다르게 만들 수 있으므로, 상황에 따라 적절하게 선택하는 것이 중요합니다.

- 정규화
    L1 정규화와 L2 정규화는 머신러닝 모델의 과적합을 방지하고 모델의 일반화 성능을 향상시키는 방법입니다. 이들은 모델의 가중치를 제한하여 모델의 복잡도를 제어하며, 가중치가 너무 커지는 것을 방지하여 노이즈에 대한 모델의 민감도를 감소시킵니다.
   -  L1 정규화 (Lasso Regularization):
        L1 정규화는 가중치 벡터의 L1 노름 (즉, 가중치의 절대값의 합)을 최소화하려고 합니다. 이는 비용 함수에 가중치의 절대값을 추가하는 형태로 이루어집니다. L1 정규화의 주요 특징 중 하나는 일부 가중치를 정확히 0으로 만들어 특성 선택의 효과를 갖는다는 것입니다. 이는 모델의 해석력을 향상시키고 불필요한 특성을 제거하여 모델을 단순화하는 데 도움이 됩니다.
   -  L2 정규화 (Ridge Regularization):
        L2 정규화는 가중치 벡터의 L2 노름 (즉, 가중치의 제곱합의 제곱근)을 최소화하려고 합니다. 이는 비용 함수에 가중치 제곱의 합을 추가하는 형태로 이루어집니다. L2 정규화는 가중치를 완전히 0으로 만들지 않지만, 모든 가중치를 작게 만듭니다. 이는 모델의 복잡도를 제한하면서도 모든 특성을 유지하게 하므로, L2 정규화는 L1 정규화보다 덜 희소한 모델을 생성합니다.

    어떤 정규화를 선택할지는 문제와 데이터에 따라 달라집니다. L1 정규화는 특성 선택이 필요하거나 희소한 모델을 원할 때 유용하며, L2 정규화는 모든 특성이 관련성이 있을 것으로 예상되는 경우에 적합합니다. 실제로는 두 방법을 결합한 Elastic Net과 같은 방법을 사용하기도 합니다.

In [10]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

params = {"penalty" : ["l2", "l1"],
         "C" : [0.01, 0.1, 1, 1, 5, 10]}

grid_clf = GridSearchCV(lr_clf, param_grid = params, scoring = "accuracy", cv = 3)
grid_clf.fit(data_scaled, cancer.target)

print(f"최적 하이퍼 파라미터 : {grid_clf.best_params_}, 최적 평균 정확도 : {grid_clf.best_score_:.4f}")

최적 하이퍼 파라미터 : {'C': 1, 'penalty': 'l2'}, 최적 평균 정확도 : 0.9754


In [11]:
best_grid_clf = grid_clf.best_estimator_
pred1 = best_grid_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred1)
print(f"로지스틱 예측 정확도 : {accuracy:.4f}")

로지스틱 예측 정확도 : 0.9883


## 나이브 베이즈 분류 (Naive Bayes Classification)

- 나이브 베이즈 분류기(Naive Bayes Classifier)는 베이즈의 정리를 적용한 간단한 확률적 분류 알고리즘입니다. 이 알고리즘은 모든 특성 값들이 서로 독립임을 가정하는데, 이 가정 때문에 '나이브(단순한)'라는 이름이 붙었습니다.

- 나이브 베이즈 분류기 작동 방식:

    - 각 클래스의 사전 확률을 계산합니다. 즉, 학습 데이터에서 각 클래스의 비율을 계산합니다.

    - 각 특성에 대해, 각 클래스에서 그 특성의 확률을 계산합니다. 이를 "우도(likelihood)"라고 부릅니다.

    - 새로운 데이터 포인트를 분류하려면, 모든 클래스에 대해 특성 값들의 우도를 곱하여 클래스의 사전 확률을 곱한 값(즉, 특성 값들이 주어졌을 때 클래스의 확률)을 계산합니다.

    - 마지막으로, 가장 높은 확률을 가진 클래스를 예측 클래스로 선택합니다.

- 작동 원리를 좀 더 직관적으로 설명하자면, 그것은 '투표'와 같은 원리로 이해할 수 있습니다. 각 특성이 클래스를 결정하는데 '표'를 주는 것처럼 생각해보세요.

    - 사전 확률 계산: 먼저, 훈련 데이터에서 각 클래스의 비율을 계산합니다. 이를 각 클래스의 '기본 표'라고 생각할 수 있습니다. 예를 들어, 100개의 데이터 중에 30개가 클래스 A, 70개가 클래스 B에 속한다면, 클래스 A의 사전 확률은 0.3, 클래스 B의 사전 확률은 0.7입니다.

    - 특성 별 확률 계산 (우도 계산): 각 클래스에서 각 특성의 확률을 계산합니다. 이는 각 특성이 클래스에 '표'를 주는 방식을 결정합니다. 예를 들어, '키'라는 특성이 있는데, 클래스 A에서는 '키가 큰' 사람의 비율이 높고, 클래스 B에서는 '키가 작은' 사람의 비율이 높다면, '키가 큰' 사람은 클래스 A에, '키가 작은' 사람은 클래스 B에 '표'를 줄 것입니다.

    - 후보자 별 투표 집계: 새로운 데이터 포인트가 주어졌을 때, 모든 특성이 각 클래스에 '표'를 주는 방식을 따라 '표'를 계산합니다. 즉, 모든 특성의 '표'를 모두 더하고, 그것에 사전 확률(기본 표)을 더한 것이 각 클래스의 최종 '표'가 됩니다.

    - 승리자 선정: 가장 많은 '표'를 받은 클래스가 새로운 데이터 포인트의 클래스가 됩니다.

- 이처럼, 나이브 베이즈 분류기는 각 특성이 독립적으로 각 클래스에 '표'를 주는 것처럼 작동하므로, 이를 '나이브(단순한)' 베이즈 분류기라고 부릅니다.

- 나이브 베이즈 분류기는 텍스트 분류(예: 스팸 메일 분류)에서 많이 사용되며, 계산이 간단하고 빠르며, 큰 데이터셋에 대해서도 잘 동작하는 특징을 가지고 있습니다. 하지만 나이브 베이즈의 '나이브'한 가정, 즉 모든 특성이 독립적이라는 가정은 실제 데이터에는 잘 맞지 않을 수 있으므로 이 점을 고려할 필요가 있습니다.
- 나이브 베이즈는 스팸 메일 필터, 텍스트 분류, 감정 분석, 추천 시스템 등에 광범위하게 활용되는 분류 기법입니다. 나이브 베이즈 분류에 대해서 배우기 위해서는 베이즈 정리를 먼저 알아야 합니다.

- 나이브 베이즈의 장점

    - 간단하고, 빠르며, 정확한 모델입니다.
    - computation cost가 작습니다. (따라서 빠릅니다.)
    - 큰 데이터셋에 적합합니다.
    - 연속형보다 이산형 데이터에서 성능이 좋습니다.
    - Multiple class 예측을 위해서도 사용할 수 있습니다.

- 단점
    - feature 간의 독립성이 있어야 합니다. 하지만 실제 데이터에서 모든 feature가 독립인 경우는 드뭅니다. 장점이 많지만 feature가 서로 독립이어야 한다는 크리티컬한 단점이 있습니다.
    - feature간 독립성이 있다는 말은 feature간에 서로 상관관계가 없다는 뜻입니다. 
    - X1과 X2라는 feature가 있을 때 X1이 증가하면 X2도 같이 증가한다고 합시다. 그럼 X1과 X2는 서로 상관관계가 있다고 말할 수 있고, 이는 X1과 X2가 독립성이 없다는 뜻입니다. 
    - X1과 X2가 독립성이 있으려면 X1이 증가하든 말든, X2에는 아무런 영향을 미치지 않아야 합니다. 하지만 우리가 얻을 수 있는 데이터에서는 feature간의 독립성이 항상 보장되지는 않습니다. 
    - 나이브 베이즈 모델은 feature간 독립성이 있다는 가정하에 성립되는 모델이기 때문에 실생활에서 바로 적용하기는 어려움있습니다.

### 과제1_0522.

어떤 마을 전체 사람들의 10.5%가 암 환자이고, 89.5%가 암 환자가 아닙니다. 이 마을의 모든 사람에 대해 암 검진을 실시했다고 합시다. 암 검진시 양성 판정, 음성 판정 결과가 나올 수 있습니다. 하지만 검진이 100% 정확하지는 않고 약간의 오차가 있습니다. 암 환자 중 양성 판정을 받은 비율은 90.5%, 암 환자 중 음성 판정을 받은 비율은 9.5%, 암 환자가 아닌 사람 중 양성 판정을 받은 비율은 20.4%, 암 환자가 아닌 사람 중 음성 판정을 받은 비율은 79.6%입니다. 어떤 사람이 양성 판정을 받았을 때 이 사람이 암 환자일 확률은 얼마일까요?

10.5% * 90.5% = P, P    9.5025%


10.5% * 9.5% = P, N     0.9975%


89.5% * 20.4% = N, P    18.258%


89.5% * 79.6% = N, N    71.242%



X, P = 9.5025 + 18.258 = 27.7605%


P, P% = 9.5025/27.7605 = 34.2302%

In [12]:
P_Cancer = 0.105
P_No_Cancer = 0.895

P_Positive_Cancer = 0.905
P_Negative_Cancer = 0.095
P_Positive_No_Cancer = 0.204
P_Negative_No_Cancer = 0.796

P_Positive = P_Positive_Cancer * P_Cancer + P_Positive_No_Cancer * P_No_Cancer
P_Cancer_Positive = P_Positive_Cancer * P_Cancer / P_Positive

print(f"양성 판정을 받은 사람이 암 환자일 확률 : {P_Cancer_Positive}")

양성 판정을 받은 사람이 암 환자일 확률 : 0.34230291241151994


Q. 나이브 베이즈 분류
문제 : 날씨가 overcast이고 기온이 mild일 때 경기를 할 확률은?

In [13]:
weather=['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast','Sunny','Sunny',
'Rainy','Sunny','Overcast','Overcast','Rainy']
temp=['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']

play=['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']

In [14]:
le = preprocessing.LabelEncoder()
weather_encoded = le.fit_transform(weather)
print(weather_encoded)

temp_encoded = le.fit_transform(temp)
label = le.fit_transform(play)
print("Temp : ", temp_encoded)
print("Play : ", label)

[2 2 0 1 1 1 0 2 2 1 2 0 0 1]
Temp :  [1 1 1 2 0 0 0 2 0 2 2 2 1 2]
Play :  [0 0 1 1 1 0 1 0 1 1 1 1 1 0]


In [15]:
features = zip(weather_encoded,temp_encoded)
features = list(features)
print(features)

[(2, 1), (2, 1), (0, 1), (1, 2), (1, 0), (1, 0), (0, 0), (2, 2), (2, 0), (1, 2), (2, 2), (0, 2), (0, 1), (1, 2)]


In [16]:
model = GaussianNB()

model.fit(features, label)

predicted = model.predict([[0,2]])
print("Predicted Value : ", predicted)

Predicted Value :  [1]


### 과제 1_0523
Label이 여러 개인 나이브 베이즈 : 3개의 와인 클래스 분류

In [17]:
# Scikit-learn을 활용한 나이브 베이즈 분류기 구축 

from sklearn import datasets

#Load dataset
wine = datasets.load_wine()

# print the names of the 13 features
print("Features: ", wine.feature_names)

# print the label type of wine(class_0, class_1, class_2)
print("Labels: ", wine.target_names)

Features:  ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
Labels:  ['class_0' 'class_1' 'class_2']


In [18]:
from sklearn import datasets
import pandas as pd

# wine 데이터셋 로드
wine = datasets.load_wine()

# 데이터프레임 생성
wine_df = pd.DataFrame(data=wine.data, columns=wine.feature_names)

# 타깃 변수 추가
wine_df['target'] = wine.target

# 데이터프레임 확인
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [19]:
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.3, random_state=109)

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),'\n')
print("Class report:", metrics.classification_report(y_test,y_pred))

Accuracy: 0.9074074074074074 

Class report:               precision    recall  f1-score   support

           0       0.91      0.95      0.93        21
           1       0.94      0.79      0.86        19
           2       0.88      1.00      0.93        14

    accuracy                           0.91        54
   macro avg       0.91      0.91      0.91        54
weighted avg       0.91      0.91      0.91        54

